In [1]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

NVIDIA A100-PCIE-40GB, 40960 MiB, 16716 MiB
NVIDIA A100-PCIE-40GB, 40960 MiB, 25962 MiB
NVIDIA A100-PCIE-40GB, 40960 MiB, 16616 MiB


In [73]:
# compute CLIP-space cosine-similarity distance
import torch
import os
import clip
import pathlib
import tqdm
import numpy as np
import warnings
import json
import sklearn.preprocessing
from diffusers import StableDiffusionPipeline
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from packaging import version
from shutil import rmtree

DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")

pretrained_model_name_or_path = "/root/autodl-tmp/stable_diffusion/stable-diffusion-v1-5"
original_image_dir = "/root/autodl-tmp/textual_inversion/download_data/cute-game-style"

In [74]:
clip_images_path_list = [os.path.join(original_image_dir, path) for path in os.listdir(original_image_dir)
                         if path.endswith(('.png', '.jpg', '.jpeg', '.tiff'))]
original_images_path_list = [os.path.join(original_image_dir, path) for path in os.listdir(
                             original_image_dir) if path.endswith(('.png', '.jpg', '.jpeg', '.tiff'))]
num_original_images = len(original_images_path_list)

In [75]:
class CLIPImageDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        # only 224x224 ViT-B/32 supported for now
        self.preprocess = self._transform_test(224)

    def _transform_test(self, n_px):
        return Compose([
            Resize(n_px, interpolation=Image.BICUBIC),
            CenterCrop(n_px),
            lambda image: image.convert("RGB"),
            ToTensor(),
            Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
        ])

    def __getitem__(self, idx):
        c_data = self.data[idx]
        image = Image.open(c_data)
        image = self.preprocess(image)
        return {'image': image}

    def __len__(self):
        return len(self.data)

In [76]:
def extract_all_images(images, model, device, batch_size=64, num_workers=8):
    data = torch.utils.data.DataLoader(
        CLIPImageDataset(images),
        batch_size=batch_size, num_workers=num_workers, shuffle=False)
    all_image_features = []
    with torch.no_grad():
        for b in tqdm.tqdm(data):
            b = b['image'].to(device)
            b = b.to(torch.float16)
            all_image_features.append(model.encode_image(b).cpu().numpy())
    all_image_features = np.vstack(all_image_features)
    return all_image_features

def get_clip_score(model, clip_images, original_images, device, w=1.0):
    if isinstance(clip_images, list):
        # need to extract image features
        clip_images = extract_all_images(clip_images, model, device)
    if isinstance(original_images, list):
        # need to extract image features
        original_images = extract_all_images(original_images, model, device)
    
    # as of numpy 1.21, normalize doesn't work properly for float16
    if version.parse(np.__version__) < version.parse('1.21'):
        clip_images = sklearn.preprocessing.normalize(clip_images, axis=1)
        original_images = sklearn.preprocessing.normalize(original_images, axis=1)
    else:
        warnings.warn(
            'due to a numerical instability, new numpy normalization is slightly different than' 
            'paper results. To exactly replicate paper results, please use numpy version less' 
            'than 1.21, e.g., 1.20.3.')
        clip_images = clip_images / np.sqrt(np.sum(clip_images ** 2, axis=1, keepdims=True))
        original_images = original_images / np.sqrt(np.sum(original_images ** 2, axis=1, 
                                                           keepdims=True))
    
    per = w * np.clip(np.dot(clip_images, original_images.T), 0, None)
    return np.mean(per)

In [77]:
clip_model, clip_transform = clip.load("ViT-B/32", device=DEVICE, jit=False)
clip_model.eval()

clip_features = extract_all_images(clip_images_path_list, clip_model, DEVICE, 
                                   batch_size=num_original_images, num_workers=8)
original_features = extract_all_images(original_images_path_list, clip_model, DEVICE, 
                                       batch_size=num_original_images, num_workers=8)

# Compute pair-wise Clip-space cosine similarity
final_score = get_clip_score(clip_model, clip_features, original_features, DEVICE)
print("END!!! CLIP image2image score is: {}".format(final_score))

/tmp/ipykernel_708350/2880399981.py:9: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  Resize(n_px, interpolation=Image.BICUBIC),
100%|██████████| 1/1 [00:00<00:00,  1.40it/s]

END!!! CLIP image2image score is: 0.69140625



/tmp/ipykernel_708350/2946529958.py:27: UserWarning: due to a numerical instability, new numpy normalization is slightly different thanpaper results. To exactly replicate paper results, please use numpy version lessthan 1.21, e.g., 1.20.3.
  warnings.warn(
